### Dataset

In [7]:
import pandas as pd
from auto_ts import auto_timeseries
import dill
import talib
import numpy as np
import warnings

warnings.filterwarnings('ignore')

from sklearn.model_selection import TimeSeriesSplit
from skopt import BayesSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import log_loss
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier

In [8]:
# Evito que ciertas columnas se transformen a notacion cientifica en las predicciones
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

In [9]:
columns = [
    # 'Open_time',
    # 'Close',
    'Open',
    'High',
    'Low',
    'SMA_20',
    'EMA_20',
    'Upper_Band',
    'Middle_Band',
    'Lower_Band',
    'RSI',
    'MACD',
    'Signal',
    'ADX',
    'SlowK',
    'SlowD',
    'CCI',
    'ATR',
]

### Armado y entrenamiento de un clasificador a partir de los datos originales

#### Modelo Light GBM

In [13]:
complete_dataset = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/automation/final_dataset.csv') 
classifier_dataset = complete_dataset[columns]
# classifier_dataset['Open_time'] = pd.to_datetime(classifier_dataset['Open_time'])
classifier_dataset['Tendencia'] = complete_dataset['Tendencia']

clasifier_validation = classifier_dataset[-10:]
classifier_dataset = classifier_dataset[:-10]

In [14]:
display(classifier_dataset.tail())

,Open,High,Low,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,MACD,Signal,ADX,SlowK,SlowD,CCI,ATR,Tendencia
974,7.64,7.67,7.28,7.13,7.26,7.84,7.13,6.42,52.34,0.08,0.02,17.11,82.52,81.69,53.53,0.38,Bajista
975,7.37,7.56,7.14,7.14,7.25,7.84,7.14,6.43,48.42,0.06,0.02,16.11,65.64,77.90,-6.88,0.39,Bajista
976,7.18,7.24,6.92,7.15,7.23,7.84,7.15,6.46,45.01,0.03,0.03,15.90,41.03,63.06,-110.51,0.38,Bajista
977,7.00,7.05,6.80,7.16,7.21,7.82,7.16,6.51,44.75,0.01,0.02,16.05,24.18,43.62,-135.76,0.37,Lateral
978,6.98,7.13,6.96,7.19,7.19,7.79,7.19,6.58,46.62,-0.01,0.02,15.86,20.39,28.53,-89.38,0.36,Lateral


In [15]:
classifier_dataset.shape

(979, 17)

In [16]:
X = classifier_dataset.drop(columns=["Tendencia"])
y = classifier_dataset["Tendencia"]

y = y.to_numpy().reshape(-1, 1)
onehot_encoder = OneHotEncoder()
y_one_hot = onehot_encoder.fit_transform(y)


In [17]:
scalers = {}
for column in X.columns:
    scaler = StandardScaler()
    X[[column]] = scaler.fit_transform(X[[column]])
    clasifier_validation[[column]] = scaler.transform(clasifier_validation[[column]])
    scalers[column] = scaler

In [18]:
display(y_one_hot)

<979x3 sparse matrix of type '<class 'numpy.float64'>'
	with 979 stored elements in Compressed Sparse Row format>

In [10]:
classifier = LGBMClassifier(objective='multiclass', num_class=3, metric='multi_logloss', verbosity=-1)

# Definir el espacio de búsqueda de hiperparámetros
param_space = {
    'num_leaves': (10, 500),
    'max_depth': (3, 20),
    'learning_rate': (0.001, 0.5), 
    'n_estimators': (50, 2000),
    'min_child_samples': (5, 200),
    'subsample': (0.5, 1.0),  
    'colsample_bytree': (0.5, 1.0),
    'reg_alpha': (0.0, 1.0),
    'reg_lambda': (0.0, 1.0),
    'objective': ['multiclass'],
    'metric': ['multi_logloss', 'multi_error'],
    'importance_type': ['gain', 'split'],
    'boosting_type': ['gbdt', 'dart']#, 'rf'],
}
# Definir función de pérdida para la búsqueda bayesiana
def multi_log_loss(estimator, X_test, y_test):
    y_pred = estimator.predict_proba(X_test)
    loss = log_loss(y_test, y_pred)
    return loss

# Definir validación cruzada temporal
cv = TimeSeriesSplit(n_splits=5).split(X)
# stratified_cv = StratifiedKFold(n_splits=10, shuffle=False)

# Realizar la búsqueda bayesiana de hiperparámetros
bayes_search = BayesSearchCV(
    classifier, 
    param_space, 
    scoring='neg_log_loss', 
    cv=cv,
    # cv=TimeSeriesSplit(n_splits=10),
    # cv=stratified_cv,
    n_iter=500,
    verbose=0,
    n_jobs=-1
)

# Realizar la búsqueda bayesiana
bayes_result = bayes_search.fit(X, y)

In [ ]:
# Show best results
print("Best score:", bayes_result.best_score_)
print("Best parameters:", bayes_result.best_params_)

# Entrenar el modelo con los mejores hiperparámetros
best_model = bayes_result.best_estimator_
best_model.fit(X, y)

Best score: 3.458719420188401
Best parameters: OrderedDict([('boosting_type', 'gbdt'), ('colsample_bytree', 1.0), ('importance_type', 'gain'), ('learning_rate', 0.5), ('metric', 'multi_logloss'), ('min_child_samples', 48), ('n_estimators', 340), ('n_iter', 500), ('num_leaves', 95), ('objective', 'multiclass'), ('reg_alpha', 0.0), ('reg_lambda', 0.0), ('subsample', 0.8933012014094083)])


LGBMClassifier(importance_type='gain', learning_rate=0.5,
               metric='multi_logloss', min_child_samples=48, n_estimators=340,
               n_iter=500, num_class=3, num_leaves=95, objective='multiclass',
               subsample=0.8933012014094083, verbosity=-1)

In [ ]:
import json

# Obtener los hiperparámetros y puntajes de los 5 mejores modelos
top_n_models = 5
best_params_list = []
best_scores_list = []

for i in range(min(top_n_models, len(bayes_search.cv_results_['params']))):
    best_params_list.append(bayes_search.cv_results_['params'][i])
    best_scores_list.append(bayes_search.cv_results_['mean_test_score'][i])

# Guardar los hiperparámetros de los 5 mejores modelos en un archivo JSON
with open('gbm_classifier/top_5_hyperparameters.json', 'w') as f:
    json.dump({'best_params': best_params_list, 'best_scores': best_scores_list}, f)

# O imprimir los hiperparámetros
print("Top 5 mejores modelos:")
for i in range(len(best_params_list)):
    print("Modelo", i+1)
    print("Hiperparámetros:", best_params_list[i])
    print("Puntaje:", best_scores_list[i])


Top 5 mejores modelos:
Modelo 1
Hiperparámetros: OrderedDict([('boosting_type', 'dart'), ('colsample_bytree', 0.8941882029766676), ('importance_type', 'gain'), ('learning_rate', 0.019882422470529012), ('metric', 'multi_error'), ('min_child_samples', 181), ('n_estimators', 418), ('n_iter', 295), ('num_leaves', 175), ('objective', 'multiclass'), ('reg_alpha', 0.6706462037686927), ('reg_lambda', 0.8272135833591839), ('subsample', 0.7914163215472552)])
Puntaje: 1.0693452836731776
Modelo 2
Hiperparámetros: OrderedDict([('boosting_type', 'gbdt'), ('colsample_bytree', 0.5589694270943428), ('importance_type', 'split'), ('learning_rate', 0.2174955782444395), ('metric', 'multi_error'), ('min_child_samples', 94), ('n_estimators', 125), ('n_iter', 94), ('num_leaves', 117), ('objective', 'multiclass'), ('reg_alpha', 0.21624249202880214), ('reg_lambda', 0.4104174649836443), ('subsample', 0.9810203155226598)])
Puntaje: 1.0427473702060328
Modelo 3
Hiperparámetros: OrderedDict([('boosting_type', 'dart'

#### Armado del ensamble

In [ ]:
# Crear una lista para almacenar los modelos individuales
individual_models = []

# Entrenar los modelos individuales
for params_str in best_params_list:
    model = LGBMClassifier(verbosity=-1, **params_str)
    model.fit(X, y)
    individual_models.append(model)

voting_model = VotingClassifier(estimators=[('model_'+str(i), model) for i, model in enumerate(individual_models)], voting='hard', verbose=0)#voting='soft', verbose=0)
voting_model.fit(X, y)

with open('gbm_classifier/gbm_boosting_classifier.pkl', 'wb') as f:
    dill.dump(voting_model, f)

#### Clasificacion con el ensamble sobre las redicciones de los modelos generativos

In [19]:
with open('gbm_classifier/gbm_boosting_classifier.pkl', 'rb') as f:
    voting_model = dill.load(f)

In [20]:
voting_model.fit(X, y)

VotingClassifier(estimators=[('model_0',
                              LGBMClassifier(colsample_bytree=0.807046987702391,
                                             learning_rate=0.23775936097872807,
                                             max_depth=10,
                                             metric='multi_logloss',
                                             min_child_samples=195,
                                             n_estimators=283, num_leaves=56,
                                             objective='multiclass',
                                             reg_alpha=0.7976862408661672,
                                             reg_lambda=0.6014168527190339,
                                             subsample=0.9443097641729721,
                                             verbosity=-1)),
                             ('model_1',...
                                             verbosity=-1)),
                             ('model_4',
                              LGBMClassifier(boosting_type='dart',
                                             colsample_bytree=0.5765094870251098,
                                             learning_rate=0.25816764889639127,
                                             max_depth=18, metric='multi_error',
                                             min_child_samples=142,
                                             n_estimators=1965, num_leaves=304,
                                             objective='multiclass',
                                             reg_alpha=0.69889705819886,
                                             reg_lambda=0.5626221846637087,
                                             subsample=0.5216632563729354,
                                             verbosity=-1))],
                 verbose=0)

##### Datos originales:

In [21]:
display(clasifier_validation)

,Open,High,Low,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,MACD,Signal,ADX,SlowK,SlowD,CCI,ATR,Tendencia
979,-0.41,-0.42,-0.40,-0.41,-0.42,-0.45,-0.41,-0.36,-0.22,0.16,0.20,-1.14,-0.96,-0.97,-0.89,-0.55,Lateral
980,-0.42,-0.42,-0.40,-0.41,-0.42,-0.45,-0.41,-0.35,-0.17,0.15,0.19,-1.18,-0.93,-1.00,-0.64,-0.56,Lateral
981,-0.41,-0.41,-0.40,-0.41,-0.42,-0.45,-0.41,-0.34,0.20,0.15,0.18,-1.24,-0.78,-0.92,-0.34,-0.56,Alcista
982,-0.40,-0.41,-0.37,-0.41,-0.42,-0.45,-0.41,-0.34,0.36,0.16,0.18,-1.32,-0.43,-0.74,0.12,-0.58,Lateral
983,-0.39,-0.41,-0.38,-0.41,-0.42,-0.45,-0.41,-0.34,0.12,0.16,0.18,-1.35,-0.26,-0.50,-0.12,-0.58,Lateral
984,-0.40,-0.41,-0.49,-0.41,-0.42,-0.45,-0.41,-0.35,-0.75,0.12,0.17,-1.11,-0.30,-0.34,-1.52,-0.52,Bajista
985,-0.45,-0.47,-0.46,-0.41,-0.43,-0.44,-0.41,-0.37,-1.13,0.07,0.15,-0.89,-0.64,-0.41,-1.70,-0.52,Bajista
986,-0.48,-0.48,-0.45,-0.42,-0.44,-0.44,-0.42,-0.38,-0.79,0.04,0.12,-0.68,-0.74,-0.58,-1.39,-0.54,Alcista
987,-0.46,-0.48,-0.46,-0.42,-0.44,-0.45,-0.42,-0.39,-0.69,0.05,0.16,-0.89,-0.83,-0.76,-1.09,-0.55,Lateral
988,-0.46,-0.48,-0.47,-0.43,-0.45,-0.45,-0.43,-0.40,-0.85,0.02,0.13,-0.23,-0.80,-0.81,-1.04,-0.55,Bajista


In [22]:
n_clases = 3 

validation_predictions = voting_model.predict(clasifier_validation[columns])
display(validation_predictions)

array(['Lateral', 'Lateral', 'Alcista', 'Lateral', 'Lateral', 'Bajista',
       'Bajista', 'Lateral', 'Lateral', 'Bajista'], dtype=object)

##### Datos generados por auto ml con Auto-TS:

In [23]:
auto_ts_feature_predictor_df = pd.read_csv('../automation/auto_ts_models/result.csv')
features_auto_ts = auto_ts_feature_predictor_df[columns]

for column in features_auto_ts.columns:
    scaler = scalers[column]
    features_auto_ts[[column]] = scaler.transform(features_auto_ts[[column]])


predictions_ts = voting_model.predict(features_auto_ts)

display(predictions_ts)

array(['Bajista', 'Bajista', 'Bajista', 'Bajista', 'Bajista', 'Bajista',
       'Bajista', 'Lateral', 'Lateral', 'Lateral'], dtype=object)

##### Datos generados por auto ml con prophet:

In [ ]:
auto_ml_prophet_df = pd.read_csv('auto_timeseries_models_prophet/predicciones.csv')
auto_mp_prophet_predictions = voting_model.predict(auto_ml_prophet_df[columns].drop(columns=["Open_time"]))
display(auto_mp_prophet_predictions)

KeyError: "['MACD', 'Signal', 'ADX', 'SlowK', 'SlowD', 'CCI', 'ATR'] not in index"

##### Datos generados por auto ml con stats:

In [ ]:
auto_ml_stats_df = pd.read_csv('auto_timeseries_models/predicciones.csv')
auto_mp_stats_predictions = voting_model.predict(auto_ml_stats_df[columns].drop(columns=["Open_time"]))

display(auto_mp_stats_predictions)

array(['Bajista', 'Bajista', 'Bajista', 'Bajista', 'Bajista', 'Bajista',
       'Bajista', 'Bajista', 'Bajista', 'Bajista'], dtype=object)

##### Datos generados por auto ml con modelos clasicos:

In [ ]:
auto_ml_df = pd.read_csv('h2o_models/predicciones.csv')
auto_mp_predictions = voting_model.predict(auto_ml_df[columns].drop(columns=["Open_time"]))
display(auto_mp_predictions)

array(['Bajista', 'Bajista', 'Bajista', 'Bajista', 'Bajista', 'Bajista',
       'Bajista', 'Bajista', 'Bajista', 'Bajista'], dtype=object)

##### Datos generados por auto ml con skforecast:

In [ ]:
skforecast_df = pd.read_csv('skforecast/predicciones.csv')
skforecast_predictions = voting_model.predict(skforecast_df[columns[1:]])
display(skforecast_predictions)

array(['Bajista', 'Lateral', 'Alcista', 'Alcista', 'Alcista', 'Alcista',
       'Alcista', 'Alcista', 'Alcista', 'Alcista'], dtype=object)